# 1. Scale Model

In [1]:
import os
import numpy as np
import opensim as osim

## Inputs

In [2]:
# root directory of the opensim project
root = '/home/romain/Downloads/opensim/'

# participant
participant = 'SylGaud'

# model file name
model_name = 'model.osim'

# static trial name
static_trial_name = 'SylGaud0.trc'

# scaling configuration
scaling_conf_name = 'scaling.xml'

In [3]:
model = {
    'name': model_name,
    'path': os.path.join(root, 'metadata', 'generic', model_name),
    'output': os.path.join(root, 'metadata', participant, 'scaled-{}'.format(model_name))
}

static_trial = {
    'name': static_trial_name,
    'path': os.path.join(root, 'outputs', participant, '0_trc', static_trial_name)
}

scaling_conf = {
    'name': scaling_conf_name,
    'path': os.path.join(root, 'metadata', 'generic', scaling_conf_name),
    'output': os.path.join(root, 'metadata', participant, 'scaled-{}'.format(scaling_conf_name))
}

IK_static = os.path.join(root, 'outputs', participant, '2_inverse_kinematic', 'static_Q.mot')

## Scaling

In [4]:
anthropo = {
    'mass': 80,
    'height': 1800,
    'age': 27
}

In [5]:
# 0. load and define model
my_model = osim.Model(model['path'])

# 1. get time range from static trial
marker_data = osim.MarkerData(static_trial['path'])
initial_time = marker_data.getStartFrameTime()
final_time = marker_data.getLastFrameTime()
range_time = osim.ArrayDouble()
range_time.set(0, initial_time)
range_time.set(1, final_time)

# 2. scaling tool
scale_tool = osim.ScaleTool(scaling_conf['path'])  # TODO edit measurment set
scale_tool.setSubjectMass(anthropo['mass'])
scale_tool.setSubjectHeight(anthropo['height'])
scale_tool.setSubjectAge(anthropo['age'])

# 3. GenericModelMarker
scale_tool.getGenericModelMaker().setModelFileName(model['path'])  # Tell scale tool to use the loaded model

# 4. ModelScaler
model_scaler = scale_tool.getModelScaler()
model_scaler.setApply(True)  # Whether or not to use the model scaler during scale
model_scaler.setMarkerFileName(static_trial['path'])  # Set the marker file to be used for scaling
model_scaler.setTimeRange(range_time)
model_scaler.setPreserveMassDist(True)  # Indicating whether or not to preserve relative mass between segments
model_scaler.setOutputModelFileName(model['output'])  # Name of model file (.osim) to write when done scaling
#scale_tool.getModelScaler().setOutputScaleFileName()  # Filename to write scale factors that were applied to the unscaled model (optional)

# 5. run model scaler tool
model_scaler.processModel(my_model)

# 6. MarkerPlacer
scaled_model = osim.Model(model['output'])  # Load a scaled model
marker_placer = scale_tool.getMarkerPlacer()  
marker_placer.setApply(True) # Whether or not to use the model scaler during scale
marker_placer.setTimeRange(range_time)
marker_placer.setStaticPoseFileName(static_trial['path'])
marker_placer.setOutputModelFileName(model['output'].replace('model', 'model-MKR'))  # Name of model file (.osim) to write when done scaling
marker_placer.setOutputMotionFileName(IK_static)  # Set the output motion filename
marker_placer.setMaxMarkerMovement(-1.00000000)  # Maximum amount of movement allowed in marker data when averaging

# 7. run MarkerPlacer
marker_placer.processModel(scaled_model)

# 8. print scale config into xml
scale_tool.printToXML(scaling_conf['output'])

True

In [6]:
# setup scale tool
scale_tool = osim.ScaleTool(scaling_conf['path'])
scale_tool.setName('scaled-{}'.format(model['name'].replace('.osim', '')))
scale_tool.getGenericModelMaker().setModelFileName(model['path'])

# get initial and final time
marker_data = osim.MarkerData(static_trial['path'])
initial_time = marker_data.getStartFrameTime()
final_time = marker_data.getLastFrameTime()
range_time = osim.ArrayDouble()
range_time.set(0, initial_time)
range_time.set(1, final_time)

# model scaler
model_scaler = scale_tool.getModelScaler()
model_scaler.setMarkerFileName(static_trial['path'])
model_scaler.setTimeRange(range_time)
model_scaler.setOutputModelFileName(model['output'])
model_scaler.setOutputScaleFileName(scaling_conf['output'])

# participant characteristics
scale_tool.setSubjectMass(participant['mass'])
scale_tool.setSubjectHeight(participant['height'])
scale_tool.setSubjectAge(participant['age'])

# run model scaler
gen_model = osim.Model(model['path'])
model_scaler.processModel(gen_model)

# IK
marker_placer = scale_tool.getMarkerPlacer()
marker_placer.setStaticPoseFileName(static_trial['path'])
marker_placer.setTimeRange(range_time)
marker_placer.setOutputModelFileName(model['output'].replace('model', 'model-MKR'))
scaled_model = osim.Model(model['output'])
#state = scaled_model.initSystem()
marker_placer.processModel(scaled_model)

scale_tool.printToXML(scaling_conf['output'])

True